In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate

from lmfit import Parameters, Minimizer, minimize, report_fit

import scqubits as scq

scq.settings.FUZZY_SLICING = True

In [3]:
osc = scq.Oscillator(E_osc = 1)
gq = scq.GenericQubit(E = 1)

g = 0.1
hilbertspace = scq.HilbertSpace([gq, osc])

hilbertspace.add_interaction(
    g_strength = g,
    op1 = osc.annihilation_operator,
    op2 = gq.sp_operator,
    add_hc = True
)

In [4]:
def update_hilbertspace(E, detuning):
    gq.E = E
    osc.E_osc = E + detuning

x = np.linspace(0, 10, 201)
egv = 4

pname1 = 'E'
E_vals = x
pname3 = 'detuning'
detuning_vals = x

paramvals_by_name = {pname1: E_vals, pname3: detuning_vals}

sweep = scq.ParameterSweep(
    hilbertspace=hilbertspace,
    paramvals_by_name=paramvals_by_name,
    update_hilbertspace=update_hilbertspace,
    evals_count=egv,
    num_cpus=4
)

Parallel compute bare eigensys [num_cpus=4]

Parallel compute bare eigensys [num_cpus=4]

Parallel compute dressed eigensys [num_cpus=4]

In [5]:
def create_model_data(x, e, transition_list):
    full_data = create_interpolated_data(x, egv, e)
    
    model = np.array([])
    for transition in transition_list:
        i = transition[0]
        f = transition[1]
        initial_state = full_data[:,i]
        final_state = full_data[:,f]
        energy_diff = final_state - initial_state
        model = np.append(model, energy_diff)
    return model

def create_interpolated_data(x, egv, e):
    egv_x = np.linspace(0, egv-1, egv)
    points = (x, x, egv_x)
    values = sweep["evals"]
    
    matrix = np.zeros((x.size,4))
    row_index = 0
    for i in x:
        column_index = 0
        for j in egv_x:
            matrix[row_index, column_index] = interpolate.interpn(points, values, np.array([e, i, j]))[0]
            column_index+=1
        row_index+=1
    return matrix

In [6]:
def jc_resid(pars, x, transition_list, data=None):
    e = pars['e']
    
    model = create_model_data(x, e, transition_list)
    if data is None:
        return model
    resid = model - data
    return resid.flatten()

params = Parameters()
params.add('e', value=3.4, min = 0, max = 10)

transitions = [(0,1),(1,2),(2,3),(0,3)]
e = 8.1
jc_data = create_model_data(x, e, transitions)
jc_noise = jc_data + 0.25*np.random.normal(size=jc_data.shape)

min1 = Minimizer(jc_resid, params, fcn_args=(x, transitions,), fcn_kws={'data': jc_noise})
out1 = min1.leastsq()
fit1 = jc_resid(out1.params, x, transitions)
report_fit(out1.params)

out2 = minimize(jc_resid, params, args=(x, transitions, jc_noise))
fit2 = jc_resid(out2.params, x, transitions)
report_fit(out2.params)

[[Variables]]
    e:  8.10894945 +/- 0.00718156 (0.09%) (init = 3.4)
[[Variables]]
    e:  8.10894945 +/- 0.00718156 (0.09%) (init = 3.4)


In [7]:
array_shape = (x.size, egv)
a = np.reshape(jc_noise, array_shape, 'F')
b = np.reshape(fit1, array_shape, 'F')
c = np.reshape(fit2, array_shape, 'F')

In [ ]:
plt.plot(x, a, 'ro')
plt.plot(x, b, 'b')
plt.plot(x, c, 'y')
plt.show()